In [2]:

import os
os.environ['KERAS_BACKEND'] = 'theano'

import pandas as pd
import json
import spacy
import textacy
import re
from textacy.preprocessing import remove_punctuation, replace_emojis, replace_urls
import textacy.vsm 
from textacy.vsm import Vectorizer
import textacy.tm
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
import os
from textblob import TextBlob
import multiprocessing as mp
import ast
from twitteremotion.emotion_predictor import EmotionPredictor
import preprocessor as p
import json
p.set_options(p.OPT.URL, p.OPT.EMOJI)


Using Theano backend.


In [15]:
path = "rawDAta/twitter_Oct20-Dec25_geocoded_Feb10YH.csv"
outPath = "processedData/"

In [16]:
df = pd.read_csv(path,encoding="iso-8859-1")

In [20]:
len(df)

762

In [21]:
other_stops = ["t","co","s", "t co","g",'s   t co',"https","htt"]

In [22]:
def getEntities(tweet):
  blob = TextBlob(tweet)
  result = []
  for sent in blob.sentences:
    sent = str(sent)
    sentence = Sentence(sent)
    tagger.predict(sentence)
    entities = sentence.to_dict(tag_type='ner')
    for ent in entities["entities"]:
      result.append([ent["text"],ent["labels"]])
  return result

In [23]:
def clean_text(text):
  text = p.clean(text)
  text = remove_punctuation(text)
  text = replace_emojis(text)
  text = replace_urls(text)
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',"",text)
  text = re.sub(r'[\d-]',"",text)
  return text

In [24]:
df["clean_text"] = df["text"].apply(clean_text)

In [25]:
emot_model = EmotionPredictor(classification='ekman', setting='mc')

/Users/darts/Desktop/karduni/phproto/pipeline/venv/lib/python3.8/site-packages/h5py/__init__.py:72: UserWarning: h5py is running against HDF5 1.10.4 when it was built against 1.8.4, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [26]:
predictions = emot_model.predict_probabilities(df["text"])

In [27]:
for emotion in ["Anger",	"Disgust",	'Fear',	"Joy",	"Sadness",	"Surprise"]:
  df[emotion] = predictions[emotion]

Entity

In [28]:
tagger = SequenceTagger.load('ner')

2021-02-16 22:39:05,755 loading file /Users/darts/.flair/models/en-ner-conll03-v0.4.pt


In [30]:
df.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,quoted_status_permalink,extended_tweet,withheld_in_countries,clean_text,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,2020-10-15 15:12:20+00:00,1.320000e+18,1.320000e+18,@feverist_ à¸à¸¥à¸±à¸à¸¢à¸±à¸,"[11, 18]","<a href=""http://twitter.com/download/iphone"" r...",False,1.320000e+18,1.320000e+18,6.920813e+07,...,NaN,NaN,NaN,feverist,0.005877,0.002119,0.019198,0.747915,0.027629,0.197262
1,2020-10-15 15:48:56+00:00,1.320000e+18,1.320000e+18,@gulfeverindia @gulfkanawut ðà¸£à¸à¸à¸§à¸...,"[28, 48]","<a href=""http://twitter.com/download/iphone"" r...",False,1.320000e+18,1.320000e+18,1.230000e+18,...,NaN,NaN,NaN,gulfeverindia gulfkanawut,0.010276,0.002954,0.022733,0.553315,0.045737,0.364985
2,2020-10-15 15:49:56+00:00,1.320000e+18,1.320000e+18,@gulfeverindia @gulfkanawut Please Can you del...,"[28, 58]","<a href=""http://twitter.com/download/iphone"" r...",False,1.320000e+18,1.320000e+18,1.230000e+18,...,NaN,NaN,NaN,gulfeverindia gulfkanawut Please Can you del...,0.097684,0.049475,0.452972,0.048981,0.217681,0.133206
3,2020-10-15 16:38:45+00:00,1.320000e+18,1.320000e+18,@A79B97G97 @gulfeverindia @gulfkanawut à¹à¸«à...,"[39, 63]","<a href=""http://twitter.com/download/iphone"" r...",False,1.320000e+18,1.320000e+18,3.490435e+09,...,NaN,NaN,NaN,ABG gulfeverindia gulfkanawut,0.011113,0.003875,0.040625,0.582208,0.031821,0.330358
4,2020-10-14 05:55:16+00:00,1.320000e+18,1.320000e+18,"Thailand reports 9 new Covid-19 cases, quarant...",NaN,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,Thailand reports new Covid cases quarantine...,0.026948,0.018163,0.122829,0.285686,0.036117,0.510258


In [31]:
# ents = [getEntities(text) for text in df["text"]]
ents= []
for i,text in enumerate(df["text"]):
    ents.append(getEntities(text))
    if i%50==0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750


In [32]:
df["ents"] = ents

In [33]:
AllLocEnts = []
for entList in df["ents"]:
  locEnts = []
  for ent in entList:
    entText = ent[0]
    entLabel = ent[1][0].to_dict()
    if entLabel["value"] == "LOC":
      locEnts.append(entText)
  AllLocEnts.append(locEnts)

Topic

In [34]:
df["locEnts"] = AllLocEnts

In [35]:
df["entity"] = df["locEnts"].apply(lambda x: x[0] if len(x)>0 else "")

In [36]:
num = 3
en = textacy.load_spacy_lang("enn") #, disable=("parser",)

In [37]:
corpus = textacy.corpus.Corpus(lang=en, data=[i for i in df["clean_text"]]) #data=texts

In [38]:
tokenized_docs = (doc._.to_terms_list(entities=False, as_strings=True, normalize = "lower") for doc in corpus) #  
tokenized_docs = ([i for i in token_list if i not in other_stops and len(i)>2] for token_list in tokenized_docs)
# do tf-idf
vectorizer = Vectorizer(norm="l2", apply_idf=True, max_df=0.95) # , vocabulary_terms = s, max_df=0.95, min_df=5, norm="l2", apply_idf=True, 
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)

In [39]:
model = textacy.tm.TopicModel("nmf", n_topics=20)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
doc_topic_matrix.shape
topicTermsDict = {}
for topic_idx, top_terms in model.top_topic_terms(vectorizer.id_to_term, top_n=20):
    topicTermsDict[topic_idx] = list(top_terms)

In [40]:
doc_topics = list(model.top_doc_topics(doc_topic_matrix=doc_topic_matrix,docs=-1,top_n=1))

In [41]:
df["topic_id"] = [i[1][0] for i in doc_topics]

In [42]:
topicTermsDict

{0: ['covid',
  'coronavirus',
  'mask',
  'tnamcot',
  'timeli',
  'japan covid',
  'vaccine',
  'gym',
  'gulfkanawut',
  'gusta',
  'gusta tu',
  '||years ago',
  'gulfeverindia',
  'h nam',
  'h ngay',
  'h ngay c',
  'h ni',
  'h ni ghi',
  'h ni lp',
  'h ni nghi'],
 1: ['feverist',
  'wori',
  'aoftronaut',
  'center',
  'perfect',
  'iamipho',
  'youtuber',
  'viewskk',
  'panya',
  'bancha panya',
  'bancha',
  'misakasiamdream',
  'ittisakg',
  'paipw',
  'suddenota',
  'joysitck',
  'writenoonz',
  'aoftronaut matsuri',
  'matsuri',
  'marclamonthill'],
 2: ['timeline',
  '||years ago',
  'gulfeverindia',
  'gusta',
  'gusta tu',
  'gym',
  'gym need',
  'h nam',
  'h ngay',
  'h ngay c',
  'h ni',
  'h ni ghi',
  'h ni lp',
  'h ni nghi',
  'ha bnh',
  'ha bnh v',
  'ha dejado',
  'gulfkanawut',
  'guidelines with world',
  'ha v'],
 3: ['feverth',
  'ifyouwantme',
  'spamfever',
  'plyfever',
  'want',
  'views',
  'baimonfever',
  'siammatsuri',
  'shot',
  'undergroundfe

In [43]:
df.to_csv(outPath+"dataWithEmotionEntityTopic.csv")
with open(outPath+"topicTermDict.json","w") as jsonFile:
    jsonFile.write(json.dumps(topicTermsDict))